In [ ]:
import os
import numpy as np

In [ ]:
ex_name = "BM_gam=0.5_N=500_d=1_D=100"
folder = os.path.join("/home", "soham", "Simulations", "spectral-NN",ex_name)
print(folder)

In [ ]:
wt_fun = "Parzen"
files = os.listdir(os.path.join(folder,wt_fun,"Results_M_L"))
files.sort()
print(files)

In [ ]:
file = files[2]
print(file)

In [ ]:
method = file.strip(".txt").split("_")
print(method)

In [ ]:
del method[:3]
print(method)
del method[-2:]
print(method)
M, L = method
print(M)
print(L)

In [ ]:
for q in qs:
    print(q)
    files_q = []
    for file in files:
        if file.strip(".txt").split("_")[2] == q:
            files_q.append(file)
    print(files_q)
    for file in files_q:
        method = file.strip(".txt").split("_")
        del method[0]
        del method[1]
        method = "_".join(method)
        print(method)